In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
# import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import tensorflow
import matplotlib.pyplot as plt
%matplotlib inline

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWa

In [2]:
SEED = constants.SEED
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tensorflow.set_random_seed(constants.SEED)
tensorflow.compat.v1.set_random_seed(constants.SEED)
print(SEED)
# torch.manual_seed(SEED)
# torch.use_deterministic_algorithms(True)


21


In [3]:
# missingness_levels = [0.1, 0.2, 0.3]

# for level in missingness_levels:
# # for level in missingness_levels[:1]:
#     train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_missingness_{level}.csv')
#     train_df = train_df.fillna(-1)
# #     print(len(train_df))
#     X_train = train_df.iloc[:, 0:-1]
#     y_train = train_df.iloc[:, -1]
#     X_train, y_train = np.array(X_train), np.array(y_train)
# #     print(X_train.shape, y_train.shape)
    
#     for steps in [int(15e6), int(12e6), int(10e6)]:
# #     for steps in [int(2e3)]:
#         dqn_model = utils.stable_dueling_ddqn(X_train, y_train, steps, True, 
#         f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/seed_{SEED}/dueling_ddqn_pr_missingness_{level}_{steps}')

In [4]:
sizes = [0.5, 0.1]
for size in sizes:
    train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_basic_{size}_seed_{SEED}.csv')
    train_df = train_df.fillna(-1)
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)
    for steps in [int(15e6)]:
        dqn_model = utils.stable_dueling_ddqn(X_train, y_train, steps, True, 
        f'../../../anemia_ml4hc/models/sb/new_different_training_sizes/{size}/dueling_ddqn_pr_seed_{SEED}_{steps}')

using just stable baselines (not 3) - dueling double dqn
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.


Invalid MIT-MAGIC-COOKIE-1 key/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning:

stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.



--------------------------------------
| % time spent exploring  | 98       |
| episodes                | 10000    |
| mean 100 episode reward | -0.7     |
| steps                   | 26379    |
| success rate            | 0.19     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 96       |
| episodes                | 20000    |
| mean 100 episode reward | -0.8     |
| steps                   | 52766    |
| success rate            | 0.1      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 94       |
| episodes                | 30000    |
| mean 100 episode reward | -0.7     |
| steps                   | 80025    |
| success rate            | 0.16     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 93       |
| episodes                | 40000    |
| mean 100 episode reward | -0.8     |
| steps                  

--------------------------------------
| % time spent exploring  | 37       |
| episodes                | 320000   |
| mean 100 episode reward | -0.9     |
| steps                   | 984877   |
| success rate            | 0.12     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 35       |
| episodes                | 330000   |
| mean 100 episode reward | -0.8     |
| steps                   | 1022173  |
| success rate            | 0.13     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 33       |
| episodes                | 340000   |
| mean 100 episode reward | -0.9     |
| steps                   | 1057865  |
| success rate            | 0.12     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 30       |
| episodes                | 350000   |
| mean 100 episode reward | -1       |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 630000   |
| mean 100 episode reward | 0        |
| steps                   | 2199858  |
| success rate            | 0.52     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 640000   |
| mean 100 episode reward | 0        |
| steps                   | 2238615  |
| success rate            | 0.51     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 650000   |
| mean 100 episode reward | -0.2     |
| steps                   | 2277233  |
| success rate            | 0.44     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 660000   |
| mean 100 episode reward | 0.1      |
| steps                  

...

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1860000  |
| mean 100 episode reward | 0.5      |
| steps                   | 7566207  |
| success rate            | 0.85     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1870000  |
| mean 100 episode reward | 0.5      |
| steps                   | 7615786  |
| success rate            | 0.83     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1880000  |
| mean 100 episode reward | 0.6      |
| steps                   | 7665243  |
| success rate            | 0.85     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1890000  |
| mean 100 episode reward | 0.5      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2170000  |
| mean 100 episode reward | 0.5      |
| steps                   | 9109222  |
| success rate            | 0.82     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2180000  |
| mean 100 episode reward | 0.5      |
| steps                   | 9159668  |
| success rate            | 0.81     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2190000  |
| mean 100 episode reward | 0.6      |
| steps                   | 9209725  |
| success rate            | 0.89     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2200000  |
| mean 100 episode reward | 0.4      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2480000  |
| mean 100 episode reward | 0.4      |
| steps                   | 10667476 |
| success rate            | 0.82     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2490000  |
| mean 100 episode reward | 0.5      |
| steps                   | 10717383 |
| success rate            | 0.87     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2500000  |
| mean 100 episode reward | 0.5      |
| steps                   | 10767868 |
| success rate            | 0.86     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2510000  |
| mean 100 episode reward | 0.5      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2790000  |
| mean 100 episode reward | 0.6      |
| steps                   | 12221959 |
| success rate            | 0.86     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2800000  |
| mean 100 episode reward | 0.5      |
| steps                   | 12270126 |
| success rate            | 0.89     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2810000  |
| mean 100 episode reward | 0.5      |
| steps                   | 12319874 |
| success rate            | 0.82     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2820000  |
| mean 100 episode reward | 0.7      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3100000  |
| mean 100 episode reward | 0.6      |
| steps                   | 13773642 |
| success rate            | 0.87     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3110000  |
| mean 100 episode reward | 0.6      |
| steps                   | 13822682 |
| success rate            | 0.86     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3120000  |
| mean 100 episode reward | 0.3      |
| steps                   | 13872558 |
| success rate            | 0.82     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3130000  |
| mean 100 episode reward | 0.5      |
| steps                  

In [ ]:
# noise_levels = [0.1, 0.2, 0.3]

# for level in noise_levels:
# # for level in noise_levels[:1]:
#     train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_noisiness_{level}.csv')
# #     print(len(train_df))
#     X_train = train_df.iloc[:, 0:-1]
#     y_train = train_df.iloc[:, -1]
#     X_train, y_train = np.array(X_train), np.array(y_train)
#     print(X_train.shape, y_train.shape)
    
#     for steps in [int(15e6), int(12e6), int(10e6)]:
# #     for steps in [int(2e3)]:
#         dqn_model = utils.stable_vanilla_dqn(X_train, y_train, steps, True, 
#                     f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/dqn_pr_noisiness_{level}_{steps}')#,

#### The Data

In [ ]:
level = 0.3
train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_missingness_{level}.csv')
train_df = train_df.fillna(-1)
train_df.head()

In [ ]:
len(train_df)

In [ ]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

In [ ]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

#### Training 

In [ ]:
# %%time
# timesteps = int(1e6)
# dqn_model = utils.stable_dqn(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
# missingness
# if level==0.1:
#     steps = int(15e6)
# elif level==0.2:
#     steps = int(10e6)
# elif level==0.3:
#     steps = int(12e6)
# else:
#     print('UNKNOWN LEVEL')
# level, steps

In [ ]:
# #noisiness
# if level==0.1:
#     steps = int(10e6)
# elif level==0.2:
#     steps = int(12e6)
# elif level==0.3:
#     steps = int(10e6)
# else:
#     print('UNKNOWN LEVEL')
# level, steps
# steps = int(12e6)
level, SEED

In [ ]:
# for steps in [int(50e6)]:
#     dqn_model = utils.stable_dqn(X_train, y_train, steps, True, 
#                                  f'../../../anemia_ml4hc/models/complete_models/dueling_ddqn_{steps}',
#                                  checkpoint_folder = '../../../anemia_ml4hc/models/sb/dueling_double_dqn',
#                                  checkpoint_prefix='dddqn_basic')
    
# for steps in [int(15e6)]:
# # for steps in [int(2e3)]:
#     dqn_model = utils.stable_double_dqn(X_train, y_train, steps, True, 
#                                  f'../../../anemia_ml4hc/models/sb/seed_{SEED}/ddqn_pr_{steps}')#,
#                                  steps = int(15e6)
# dqn_model = utils.stable_dueling_dqn(X_train, y_train, steps, True, 
# f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/seed_{SEED}/dueling_dqn_pr_noisiness_{level}_{steps}')
for steps in [int(15e6), int(12e6), int(10e6)]:
    dqn_model = utils.stable_dueling_dqn(X_train, y_train, steps, True, 
    f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/seed_{SEED}/dueling_dqn_pr_missingness_{level}_{steps}')

#### Testing

#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/test_df_2800000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/success_df_2800000.csv', index=False)